# Vertices for Hubbard atom

In [ ]:
using Revise
using LinearAlgebra
using SparseIR
import SparseIR: valueim
using MSSTA
import MSSTA: QuanticsInd, quantics_to_index, asqubits, index_to_quantics, QubitInd, qubit_to_index, index_to_qubit
using OvercompleteIR
import OvercompleteIR: PHConvention, freq_box
import OvercompleteIR.Atom: HubbardAtom, MagneticChannel, chi0, full_vertex, gamma
using Plots
import TensorCrossInterpolation as TCI

newaxis = [CartesianIndex()]

In [ ]:
using ITensors

In [ ]:
U = 1.0
beta  = 3.0
ch = DensityChannel()
conv = PHConvention()
model = HubbardAtom(U, beta)

In [ ]:
# TCI with R bits
function create_func(func, R)
    # Origin of frequency box
    origin = -2^(R-1)

    m = MSSTA.InherentDiscreteGrid{3}(R, (origin, origin, origin))

    function q_to_n(q::Vector{QuanticsInd{3}})::NTuple{3, Int}
        return MSSTA.originalcoordinate(m, q)
    end

    function func_q(q::Vector{QuanticsInd{3}})::ComplexF64
        idx_3d = q_to_n(q)
        if iseven(idx_3d[1]) || iseven(idx_3d[2]) || isodd(idx_3d[3])
            return 0.0
        end
        return func((FermionicFreq(idx_3d[1]), FermionicFreq(idx_3d[2]), BosonicFreq(idx_3d[3])))
    end

    return q_to_n, func_q, m
end

R = 20
q2n, fq, mesh = create_func(w->gamma(ch, model, w), R)
;

In [ ]:
fI = x->fq(QuanticsInd{3}.(x))
localdims = fill(8, R)

firstpivot = convert.(Int, index_to_quantics((2, 2, 1), R))
#fq(QuanticsInd{3}.(firstpivot))
@show fI(firstpivot)
firstpivot = TCI.optfirstpivot(fI, localdims, firstpivot)
@show fI(firstpivot)

In [ ]:
qtt, ranks, errors = TCI.crossinterpolate2(
    ComplexF64, fI, localdims,
    [firstpivot], tolerance=1e-8, maxiter=10, verbosity=1
)

In [ ]:
err(idx) = abs(TCI.evaluate(qtt, idx) - fI(idx))

for i in 1:100
    newpivot = rand(1:8, R)
    #@show newpivot
    newpivot = TCI.optfirstpivot(fI, localdims, newpivot)
    if err(newpivot) > 1.0
        @show "Found a good pivot", err(newpivot)
        break
    end
end

In [ ]:
q_diagonal = [MSSTA.to_quantics(mesh, (2, 2, 1) .+ i) for i in 0:2^(R-5):2^R-1]
reconst_diagonal = [TCI.evaluate(qtt, convert.(Int, q)) for q in q_diagonal]
ref_diagonal = fq.(q_diagonal)
;

In [ ]:
p = plot(yaxis=:log, ylims=(1e-10,1e+4))
plot!(p, abs.(reconst_diagonal), marker=:x)
plot!(p, abs.(ref_diagonal), marker=:+)
plot!(p, abs.(ref_diagonal .- reconst_diagonal))

In [ ]:
f = collect(-101:2:101)
b = 2

is = collect(Iterators.product(f, f,  b))
ws = [MSSTA.gridpoint(mesh, i) for i in is]
qs = [MSSTA.to_quantics(mesh, w) for w in ws]

vals = [TCI.evaluate(qtt, convert.(Int, q)) for q in qs]
refs = fq.(qs)
;

In [ ]:
"""
idx_ = argmax(abs.(refs .- vals))
idx = f[idx_[1]], f[idx_[2]], b

f1(idx) = (TCI.evaluate(qtt, convert.(Int, idx)))
f2(idx) = (fI(convert.(Int, idx)))
@show idx

p = MSSTA.gridpoint(mesh, idx)
q = MSSTA.to_quantics(mesh, p)

@show convert.(Int, q)
@show f1(q)
@show f2(q)
@show err(convert.(Int, q))

#newpivot = convert.(Int, MSSTA.to_quantics(mesh, Tuple(argmax(abs.(refs .- vals)))))
#err(newpivot)
"""

In [ ]:
heatmap((abs.(vals)))

In [ ]:
heatmap((abs.(refs)))

In [ ]:
col = size(refs, 2) ÷ 2
p = plot(yaxis=:log, ylims=(1e-10,1e+4))
plot!(p, abs.(refs[:, col]), marker=:x)
plot!(p, abs.(vals[:, col]), marker=:o)
plot!(p, abs.(refs[:, col] .- vals[:, col]))

In [ ]:
heatmap(log.(abs.(vals .- refs)))

In [ ]:
plot(TCI.linkdims(qtt), yaxis=:log, marker=:x)